In [16]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

In [17]:
from finn.core.modelwrapper import ModelWrapper
from finn.builder.build_dataflow_config import (
    DataflowBuildConfig,
    ShellFlowType,
)
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC
from finn.transformation.double_to_single_float import DoubleToSingleFloat
import finn.transformation.streamline.absorb as absorb
import finn.transformation.streamline.reorder as reorder
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.streamline.collapse_repeated import CollapseRepeatedMul
from finn.transformation.remove import RemoveIdentityOps
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    GiveUniqueParameterTensors,
    RemoveUnusedTensors,
)
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.change_datalayout import ChangeDataLayoutQuantAvgPool2d
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.insert_topk import InsertTopK

In [18]:
os.environ["FINN_BUILD_DIR"]="./build"

model_name = "dwconv_test-ready"

# which platforms to build the networks for
platform_name = "ZCU104"

In [19]:
# select target clock frequency
def select_clk_period(platform):
    return 5.4

### **Customize in the following steps**

- define ownerself step
- add to ```select_build_steps```
- recall ```build_dataflow_cfg```

*If we want to start with a particular step，set ```start_step ``` *


In [20]:
def step_mobilenet_tidy(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(InferShapes())
    model = model.transform(FoldConstants())
    #model = model.transform(InsertTopK())
    #model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
    model = model.transform(InferShapes())
    model = model.transform(InferDataTypes())
    model = model.transform(InferDataLayouts())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveUniqueParameterTensors())
    model = model.transform(GiveReadableTensorNames())
    return model

In [21]:
def step_mobilenet_streamline(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(Streamline())
    additional_streamline_transformations = [
        DoubleToSingleFloat(),
        reorder.MoveMulPastDWConv(),
        absorb.AbsorbMulIntoMultiThreshold(),
        ChangeDataLayoutQuantAvgPool2d(),
        InferDataLayouts(),
        reorder.MoveTransposePastScalarMul(),
        absorb.AbsorbTransposeIntoFlatten(),
        reorder.MoveFlattenPastAffine(),
        reorder.MoveFlattenPastTopK(),
        reorder.MoveScalarMulPastMatMul(),
        CollapseRepeatedMul(),
        #RemoveIdentityOps(),
        RoundAndClipThresholds(),
    ]
    for trn in additional_streamline_transformations:
        model = model.transform(trn)
        model = model.transform(GiveUniqueNodeNames())
        model = model.transform(GiveReadableTensorNames())
        model = model.transform(InferDataTypes())
    return model

In [22]:
def step_mobilenet_lower_convs(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(LowerConvsToMatMul())
    model = model.transform(MakeMaxPoolNHWC())
    model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
    model = model.transform(MakeMaxPoolNHWC())
    model = model.transform(absorb.AbsorbConsecutiveTransposes())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    model = model.transform(InferDataTypes())
    model = model.transform(RoundAndClipThresholds())
    model = model.transform(InferDataLayouts())
    return model

In [23]:
def step_mobilenet_convert_to_hls_layers(model: ModelWrapper, cfg: DataflowBuildConfig):
    mem_mode = cfg.default_mem_mode.value
    model = model.transform(to_hls.InferPool_Batch())
    model = model.transform(to_hls.InferConvInpGen())
    model = model.transform(to_hls.InferVVAU())
    model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
    model = model.transform(to_hls.InferChannelwiseLinearLayer())
    model = model.transform(to_hls.InferLabelSelectLayer())
    model = model.transform(InferShapes())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    return model

In [24]:
def step_mobilenet_slr_floorplan(model: ModelWrapper, cfg: DataflowBuildConfig):
    if cfg.shell_flow_type == ShellFlowType.VITIS_ALVEO:
        try:
            from finn.analysis.partitioning import partition
            # apply partitioning of the model, restricting the first and last layers to SLR0
            default_slr = 0
            abs_anchors = [(0,[default_slr]),(-1,[default_slr])]
            floorplan = partition(model, cfg.synth_clk_period_ns, cfg.board, abs_anchors=abs_anchors, multivariant=False)[0]
            # apply floorplan to model
            model = model.transform(ApplyConfig(floorplan))
            print("SLR floorplanning applied")
        except:
            print("No SLR floorplanning applied")
    return model

In [25]:
def step_mobilenet_convert_to_hls_layers_separate_th(
    model: ModelWrapper, cfg: DataflowBuildConfig
):
    mem_mode = cfg.default_mem_mode.value
    model = model.transform(to_hls.InferPool_Batch())
    model = model.transform(to_hls.InferConvInpGen())
    model = model.transform(to_hls.InferThresholdingLayer())
    model = model.transform(to_hls.InferVVAU())
    model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
    model = model.transform(to_hls.InferChannelwiseLinearLayer())
    model = model.transform(to_hls.InferLabelSelectLayer())
    model = model.transform(InferShapes())
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(GiveReadableTensorNames())
    return model

### default support steps
#### PATH: finn/src/finn/builder/build_dataflow_steps.py

```python
build_dataflow_step_lookup = {
    "step_qonnx_to_finn": step_qonnx_to_finn,
    "step_tidy_up": step_tidy_up,
    "step_streamline": step_streamline,
    "step_convert_to_hls": step_convert_to_hls,
    "step_create_dataflow_partition": step_create_dataflow_partition,
    "step_target_fps_parallelization": step_target_fps_parallelization,
    "step_apply_folding_config": step_apply_folding_config,
    "step_generate_estimate_reports": step_generate_estimate_reports,
    "step_hls_codegen": step_hls_codegen,
    "step_hls_ipgen": step_hls_ipgen,
    "step_set_fifo_depths": step_set_fifo_depths,
    "step_create_stitched_ip": step_create_stitched_ip,
    "step_measure_rtlsim_performance": step_measure_rtlsim_performance,
    "step_make_pynq_driver": step_make_pynq_driver,
    "step_out_of_context_synthesis": step_out_of_context_synthesis,
    "step_synthesize_bitfile": step_synthesize_bitfile,
    "step_deployment_package": step_deployment_package,
}
```

In [26]:
def select_build_steps(platform):
    return [
        step_mobilenet_tidy,
        step_mobilenet_streamline,
        step_mobilenet_lower_convs,
        step_mobilenet_convert_to_hls_layers_separate_th,
        "step_create_dataflow_partition",
        "step_apply_folding_config",
        "step_generate_estimate_reports",
        "step_hls_codegen",
        "step_hls_ipgen",
        "step_set_fifo_depths",
        "step_create_stitched_ip",
        "step_synthesize_bitfile",
        "step_make_pynq_driver",
        "step_deployment_package",
    ]

#### Just run some special steps:

```
    #: If given, start from this step, loading the intermediate model generated
    #: from the previous step (save_intermediate_models must be enabled)
    start_step: Optional[str] = None

    #: If given, stop at this step.
    stop_step: Optional[str] = None
```

In [29]:
# start_step = 'step_mobilenet_tidy' 
# stop_step =  'step_mobilenet_convert_to_hls_layers_separate_th'

# start run step, can not set the first step.
#def_start_step = None 
def_start_step = 'step_mobilenet_streamline'

# stop run step, can be set the same with start step, it will just run one step.
#def_stop_step= None
def_stop_step =  'step_mobilenet_streamline'

In [30]:
shell_flow_type = build_cfg.ShellFlowType.VIVADO_ZYNQ
vitis_platform = None
# for Zynq, use the board name as the release name
# e.g. ZCU104
release_platform_name = platform_name
platform_dir = "release/%s" % release_platform_name
os.makedirs(platform_dir, exist_ok=True)

cfg = build_cfg.DataflowBuildConfig(
    steps=select_build_steps(platform_name),
    start_step=def_start_step,
    stop_step=def_stop_step,
    output_dir="output_%s_%s" % (model_name, release_platform_name),
    folding_config_file="folding_config/%s_folding_config.json" % platform_name,
    synth_clk_period_ns=select_clk_period(platform_name),
    board=platform_name,
    shell_flow_type=shell_flow_type,
    vitis_platform=vitis_platform,
    # folding config comes with FIFO depths already
    auto_fifo_depths=False,
    # enable extra performance optimizations (physopt)
    vitis_opt_strategy=build_cfg.VitisOptStrategyCfg.PERFORMANCE_BEST,
    generate_outputs=[
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
)
model_file = "./models/%s.onnx" % model_name
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from intermediate checkpointoutput_dwconv_test-ready_['ZCU104']/intermediate_models/step_mobilenet_tidy.onnx
Intermediate outputs will be generated in ./build
Final outputs will be generated in output_dwconv_test-ready_['ZCU104']
Build log is at output_dwconv_test-ready_['ZCU104']/build_dataflow.log
Running step: step_mobilenet_streamline [1/1]
Completed successfully


0